# Importing the required Libraries

In [20]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import exc
from sqlalchemy import inspect

# Cleaning and Transformation followed by Extracting the Dataset from CSV to a Pandas Dataframe

In [12]:
try:
    # Load the dataset
    df = pd.read_csv('Chicago latest .csv')

    # Remove any rows with missing values
    df.dropna(inplace=True)

    # Drop any columns that are not needed for analysis
    columns_to_drop = ['CASE#',' IUCR', 'X COORDINATE', 'Y COORDINATE', 'LATITUDE', 'LONGITUDE', 'LOCATION']
    df.drop(columns=columns_to_drop, inplace=True)

    # Rename columns to make them easier to work with
    new_column_names = { 'DATE  OF OCCURRENCE': 'date', 'BLOCK': 'location', ' PRIMARY DESCRIPTION': 'primary_description', ' SECONDARY DESCRIPTION': 'secondary_description', ' LOCATION DESCRIPTION': 'landmark', 'ARREST': 'arrest', 'DOMESTIC': 'domestic', 'BEAT': 'beat', 'WARD': 'ward', 'FBI CD': 'fbi_cd'}
    df.rename(columns=new_column_names, inplace=True)
    
    # Select the first 10000 rows
    df = df.head(10000)
   
     # Save the cleaned dataset to a new file
    df.to_csv('cleaned_dataset.csv', index=False)

    # Load the cleaned dataset
    df = pd.read_csv('cleaned_dataset.csv')

    # Drop all rows with null values
    df.dropna(inplace=True)

except Exception as e:
    print("Error:", e)


In [15]:
selected_columns = ['date','location', 'primary_description', 'secondary_description', 'landmark', 'arrest', 'domestic', 'beat', 'ward', 'fbi_cd']

# Create a new DataFrame with just the selected columns
vis_df = df[selected_columns].copy()


In [16]:
vis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   10000 non-null  object 
 1   location               10000 non-null  object 
 2   primary_description    10000 non-null  object 
 3   secondary_description  10000 non-null  object 
 4   landmark               10000 non-null  object 
 5   arrest                 10000 non-null  object 
 6   domestic               10000 non-null  object 
 7   beat                   10000 non-null  int64  
 8   ward                   10000 non-null  float64
 9   fbi_cd                 10000 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 781.4+ KB


In [17]:
vis_df.head()

,date,location,primary_description,secondary_description,landmark,arrest,domestic,beat,ward,fbi_cd
0,03/24/2023 04:15:00 PM,017XX W FULLERTON AVE,ASSAULT,AGGRAVATED - HANDGUN,COMMERCIAL / BUSINESS OFFICE,N,N,1931,32.0,04A
1,03/24/2023 05:08:00 PM,006XX E GRAND AVE,ASSAULT,SIMPLE,PARKING LOT / GARAGE (NON RESIDENTIAL),N,N,1834,42.0,08A
2,02/06/2023 12:00:00 AM,047XX S KOLIN AVE,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,STREET,N,N,815,14.0,11
3,11/03/2022 12:40:00 AM,034XX W LAKE ST,HOMICIDE,FIRST DEGREE MURDER,STREET,N,N,1123,28.0,01A
4,02/19/2023 04:00:00 AM,008XX E 49TH ST,CRIMINAL DAMAGE,TO PROPERTY,APARTMENT,N,Y,223,4.0,14


# Loading the claeaned dataset into PostgresSQL

In [7]:
import psycopg2


# Define your database connection details.
dbname = "dap_project"
user = "postgres"
password = "postgres"
host = "localhost"


# Connect to the database.
conn = psycopg2.connect(database=dbname,
                        user=user,
                        password=password,
                        host=host)

print("connection successfull")

connection successfull


# Creating a Table 

In [18]:
try:
    cur = conn.cursor() # Create a cursor object for executing SQL queries.
    # SQL query to create the table with column names and data types.
    cur.execute('''
        CREATE TABLE IF NOT EXISTS chicago_crime(
            date TEXT,
            location TEXT,
            primary_description TEXT,
            secondary_description TEXT,
            landmark TEXT,
            arrest TEXT,
            domestic TEXT,
            beat TEXT,
            ward TEXT,
            fbi_cd TEXT
        );
    ''')
    conn.commit() # Commit the transaction.
    print("Table created successfully!")
except psycopg2.Error as e:
    print("Error creating table:", e)
    conn.rollback() # Rollback the transaction in case of an error.
finally:
    cur.close() # Close the cursor.

Table created successfully!


# Loading the Dataframe into created Table

In [22]:
try:
    engine = create_engine(f'postgresql://{user}:{password}@{host}:5432/{dbname}')
    inspector = inspect(engine)
    # Check if the table already contains data, if so, delete all the rows. 
    table_name = 'chicago_crime'
    if table_name in inspector.get_table_names():
        with conn.cursor() as cursor:
            cursor.execute(f"DELETE FROM chicago_crime")
            conn.commit()  # Commit the DELETE statement
            #print(f"All rows deleted from table '{table_name}'.")
    vis_df.to_sql('chicago_crime', engine, if_exists='append', index=False)
    engine.dispose() # Dispose the engine after data is loaded.
    print("Data loaded to PostgreSQL successfully!")
except exc.SQLAlchemyError as e:
    print("Error loading data to PostgreSQL:", e)

Data loaded to PostgreSQL successfully!
